In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import tabula
import PyPDF2 as pdf

In [3]:
def master_doc(filename):
    frames = []
    length = pdf_page_count(filename)
    new_length = length + 1
    for x in range(3, new_length):
        df = tabula.read_pdf(filename, pandas_options={'header': 2}, pages = x, multi_tables=True, guess = False, stream=True,lattice=False)
        frames.append(df)
    full_df = pd.concat(frames)
    full_df = full_df.reset_index(drop = True)
    full_df['CUSIP NO'] = full_df['CUSIP NO'].str.replace(" ", "")
    full_df['ISSUER NAME'] = full_df["ISSUER NAME"].str.replace('*', "")
    full_df = full_df.drop(full_df.tail(1).index)
    full_df['ISSUER DESCRIPTION'] = full_df['ISSUER DESCRIPTION'].combine_first(full_df['ISSUER DESCRIPTION STATUS'])
    full_df = full_df[['CUSIP NO', 'ISSUER DESCRIPTION', 'ISSUER NAME', 'STATUS']]
    return full_df

In [4]:
def pdf_page_count(filename):
    pdfFileObj = open(filename, 'rb')
    pdfReader = pdf.PdfFileReader(pdfFileObj)
    length = pdfReader.numPages
    return length

In [5]:
df = master_doc('13flist2019q3.pdf')

of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.
 [ipykernel_launcher.py:8]


In [6]:
df.tail(20)

,CUSIP NO,ISSUER DESCRIPTION,ISSUER NAME,STATUS
17814,98980G952,PUT,ZSCALER INC,NaN
17815,98980L101,CL A,ZOOM VIDEO COMMUNICATIONS IN,NaN
17816,98980L901,CALL,ZOOM VIDEO COMMUNICATIONS IN,NaN
17817,98980L951,PUT,ZOOM VIDEO COMMUNICATIONS IN,NaN
17818,989817101,COM,ZUMIEZ INC,NaN
17819,989817901,CALL,ZUMIEZ INC,NaN
17820,989817951,PUT,ZUMIEZ INC,NaN
17821,98983V106,COM CL A,ZUORA INC,NaN
17822,98983V906,CALL,ZUORA INC,NaN
17823,98983V956,PUT,ZUORA INC,NaN


In [29]:
def equity_df(master_df):
    equity_df = master_df.loc[master_df['ISSUER DESCRIPTION'].str.contains('COM|SHS| SH|CL')]
    return equity_df

In [30]:
def debt_df(master_df):
    debt_df = master_df.loc[master_df['ISSUER DESCRIPTION'].str.contains("DEBT")]
    return debt_df

In [31]:
def option_df(master_df):
    option_df = master_df.loc[master_df['ISSUER DESCRIPTION'].str.contains("CALL|PUT")]
    return option_df

In [32]:
def warrant_df(master_df):
    warrant_df = master_df.loc[master_df['ISSUER DESCRIPTION'].str.contains("W EXP")]
    return warrant_df

In [33]:
def other_df(master_df, classified):
    other_df = master_df[~master_df.isin(classified)].dropna()
    return other_df

In [34]:
equity_df = equity_df(df)
debt_df = debt_df(df)
option_df = option_df(df)
warrant_df = warrant_df(df)

In [36]:
#equity_df.head(50)
equity_df.count()

CUSIP NO              5350
ISSUER DESCRIPTION    5350
ISSUER NAME           5350
STATUS                 304
dtype: int64

In [14]:
#debt_df.head(40)
debt_df.count()

CUSIP NO              24
ISSUER DESCRIPTION    24
ISSUER NAME           24
STATUS                 4
dtype: int64

In [15]:
#option_df.head()
option_df.count()

CUSIP NO              9103
ISSUER DESCRIPTION    9103
ISSUER NAME           9103
STATUS                 299
dtype: int64

In [16]:
#warrant_df.head()
warrant_df.count()

CUSIP NO              195
ISSUER DESCRIPTION    195
ISSUER NAME           195
STATUS                 40
dtype: int64

In [38]:
classified = [equity_df, debt_df, option_df, warrant_df]
other_df = other_df(df, classified)

In [39]:
other_df.count()

CUSIP NO              817
ISSUER DESCRIPTION    817
ISSUER NAME           817
STATUS                817
dtype: int64

In [25]:
other_df

,CUSIP NO,ISSUER DESCRIPTION,ISSUER NAME,STATUS
15,G01125106,CL A COM,AFYA LTD,ADDED
16,G01125906,CALL,AFYA LTD,ADDED
17,G01125956,PUT,AFYA LTD,ADDED
18,G0120M109,SHS,AGBA ACQUISITION LTD,ADDED
19,G0120M117,*W EXP 05/10/202,AGBA ACQUISITION LTD,ADDED
20,G0120M125,RIGHT 05/10/2024,AGBA ACQUISITION LTD,ADDED
104,G1288E103,SHS,BIQI INTL HLDGS CORP,DELETED
105,G1329V106,SHS,BLUE HAT INTERACTIVE ENTMT T,ADDED
107,G1466R207,SHS,BORR DRILLING LTD,ADDED
123,G1644T109,SHS,BRIGHTSPHERE INVESTMNT GRP P,DELETED


In [26]:
warrant_df

,CUSIP NO,ISSUER DESCRIPTION,ISSUER NAME,STATUS
11,G0080J120,*W EXP 04/30/202,ACT II GLOBAL ACQUISITION CO,NaN
19,G0120M117,*W EXP 05/10/202,AGBA ACQUISITION LTD,ADDED
52,G04415124,*W EXP 07/31/202,ANDINA ACQUISITION CORP III,NaN
95,G1117K106,*W EXP 03/14/202,BIOCERES CROP SOLUTIONS CORP,NaN
134,G18920127,*W EXP 01/01/202,CAPITOL INVT CORP IV,DELETED
161,G21810117,*W EXP 11/09/201,CLARIVATE ANALYTICS PLC,NaN
163,G22707114,*W EXP 99/99/999,COLLIER CREEK HOLDINGS,NaN
169,G2379F116,*W EXP 03/23/202,CONSTELLATION ALPHA CAP CORP,DELETED
182,G2692M111,*W EXP 10/20/202,DD3 ACQUISITION CORP,NaN
199,G2956M112,*W EXP 10/29/202,8I ENTERPRISES ACQUISITIN CO,NaN
